In [307]:
try:
  from google.colab import drive
  drive.mount('/content/drive')
  data = "/content/drive/MyDrive/ComputationalGenomics/"
except:
  data = "/Users/fionshiau/Library/CloudStorage/GoogleDrive-fioncshiau@gmail.com/My Drive/ComputationalGenomics"


In [2]:
%%capture
!pip install pomegranate;
!pip install watermark;

In [288]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style('whitegrid')
import numpy as np
import pandas as pd
import os
import glob
from hmmlearn.hmm import MultinomialHMM

from pomegranate import *

np.random.seed(0)
np.set_printoptions(suppress=True)

%load_ext watermark
%watermark -m -n -p numpy,scipy,pomegranate

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
numpy      : 1.24.2
scipy      : 1.10.1
pomegranate: 0.14.8

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 8
Architecture: 64bit



In [ ]:
assays = os.listdir(data)

In [ ]:
obs = [pd.read_table(glob.glob(os.path.join(data,i,'*binary'))[0],header = None, 
                     names= ['chr','start','end',i]) for i in assays if i!= 'README']

In [ ]:
for i in range(len(obs)):
  obs[i].index = obs[i].loc[:,'chr'].values + '_' + obs[i].loc[:,'start'].values.astype(str) + '-' + obs[i].loc[:,'end'].values.astype(str)
  

In [ ]:
obs = pd.concat(obs,axis=1)

In [ ]:
obs.columns

Index(['chr', 'start', 'end', 'H3K4me2', 'chr', 'start', 'end', 'H3K9Ac',
       'chr', 'start', 'end', 'H3K4me3', 'chr', 'start', 'end', 'ATAC-seq',
       'chr', 'start', 'end', 'H3K4me1', 'chr', 'start', 'end', 'H3K27Ac',
       'chr', 'start', 'end', 'HiC', 'chr', 'start', 'end', 'H3K27me3', 'chr',
       'start', 'end', 'RNA-Pol_ChIPseq'],
      dtype='object')

In [ ]:
obs = obs.loc[:,~obs.columns.duplicated()].copy()


In [ ]:
obs

,chr,start,end,H3K4me2,H3K9Ac,H3K4me3,ATAC-seq,H3K4me1,H3K27Ac,HiC,H3K27me3,RNA-Pol_ChIPseq
chr1_0-200,chr1,0,200,0,0,0,0,0,0,0,0,0
chr1_200-400,chr1,200,400,0,0,0,0,0,0,0,0,0
chr1_400-600,chr1,400,600,0,0,0,0,0,0,0,0,0
chr1_600-800,chr1,600,800,0,0,0,0,0,0,0,0,0
chr1_800-1000,chr1,800,1000,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
chr22_51303600-51303800,chr22,51303600,51303800,0,0,0,0,0,0,0,0,0
chr22_51303800-51304000,chr22,51303800,51304000,0,0,0,0,0,0,0,0,0
chr22_51304000-51304200,chr22,51304000,51304200,0,0,0,0,0,0,0,0,0
chr22_51304200-51304400,chr22,51304200,51304400,0,0,0,0,0,0,0,0,0


In [ ]:
obs.iloc[0:5,:]

,chr,start,end,H3K4me2,H3K9Ac,H3K4me3,ATAC-seq,H3K4me1,H3K27Ac,HiC,H3K27me3,RNA-Pol_ChIPseq
chr1_0-200,chr1,0,200,0,0,0,0,0,0,0,0,0
chr1_200-400,chr1,200,400,0,0,0,0,0,0,0,0,0
chr1_400-600,chr1,400,600,0,0,0,0,0,0,0,0,0
chr1_600-800,chr1,600,800,0,0,0,0,0,0,0,0,0
chr1_800-1000,chr1,800,1000,0,0,0,0,0,0,0,0,0


In [ ]:
os.makedirs(os.path.join("/Users/fionshiau/comp_geno",'merged'),exist_ok=True)

In [ ]:
local_path = "/Users/fionshiau/comp_geno"
obs.to_csv(os.path.join(local_path,'merged','merged_1.csv'))